### Import packages

In [1]:
# Scientific computing
import numpy as np

# Visualisation
import matplotlib.pyplot as plt

# data structure
import pandas as pd

# Import the storage tank Class
from cryoevap.storage_tanks import Tank

# Import Cryogen class
from cryoevap.cryogens import Cryogen

### Define all Variables

In [2]:
# Cylindrical storage tank properties
Q_roof = 0 # Roof heat ingress / W
T_air = 298.15 # Temperature of the environment K

# Set overall heat transfer coefficient through the walls for liquid and vapour
U_L = 4e-3 # W/m2/K
U_V = 4e-3 # W/m2/K

# Specify wall heat partitioning fraction
eta_w= 0.9

# Specify tank operating pressure
P = 100000 # Pa

### Variables of interest

# Vertically orientated cylindrical tank volume
V_tank = 40000 #m^3 | 

# Initial liquid filling / Dimensionless
LFs = [0.3, 0.95] # values of 0.95 and 0.30

# especify compound in the tank
Compound = "hydrogen" # hydrogen, methane, ammonia, nitrogen

# Especify if the data should be saved in the output.xlxs excel
sv = True

In [3]:
# Define evaporation time to simulate
hours = [24 * 30, 24 * 2] #[24 * 2, 24 * 30]
evap_times = [x * 3600 for x in hours]
# helpful variables
mults = [1 , 2 , 3]
# Thickness of the in % of the internal diameter
ST = 1.02

# Show sim duration
print(f"Simulation time: {hours} h")

Simulation time: [720, 48] h


In [4]:
# Define objective function
def BOR_function(a):
    # Update aspect ratio
    # Calculate internal diameter
    d_i = ((4 * V_tank)/(np.pi * a))**(1/3) # internal diameter / m
    d_o = d_i * ST # external diameter / m
    
    # Initialize tank
    mid_tank = Tank(d_i, d_o, V_tank, LF)

    # Set cryogen
    mid_tank.cryogen = Cryogen(name = Compound)
    
    mid_tank.cryogen.set_coolprops(P)

    # Heat flux
    q_b = mult*U_L*(T_air-mid_tank.cryogen.T_sat)  
    
    Q_b=q_b*mid_tank.A_T
    
    #Set Heat transfer propierties
    mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w)

    mid_tank.U_roof = U_V
    
    # Define vertical spacing
    # dz = 0.05 # 0.1 0.01 0.001
    # Calculate number of nodes
    
    # n_z = 1 + int(np.round(mid_tank.l_V/dz, 0))

    n_z = 20
    
    # Define dimensionless computational grid
    mid_tank.z_grid = np.linspace(0, 1, n_z)
    
    # Execute simulation
    mid_tank.evaporate(evap_time)
    
    # Calculate BOR
    BOR = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]

    print("a = %.3f, BOR=%.3e" % (a, BOR))
    
    return BOR

In [5]:
from scipy.optimize import Bounds, minimize

# Minimum and maximum practical ranges
# of the aspect ratio
bounds = Bounds([0.1], [5])

# Initial Aspect Ratio to optimise
x0 = 1 

# Solution of the optimization 
for mult in mults:
    for LF in LFs:
        for evap_time in evap_times:
            print(f"\nOptimum search for Tank Volume: {V_tank}, Liquid Filling: {LF*100}%, Compound: {Compound}, time: {evap_time/3600} hrs")
            res = minimize(BOR_function, x0, method='trust-constr',tol=1e-8 , options={'verbose': 1}, bounds=bounds)
            
            # Calculate internal diameter for optimum
            d_i = ((4 * V_tank)/(np.pi * res.x[0]))**(1/3) # internal diameter / m
            d_o = d_i * ST # external diameter / m
            
            # Initialize optimum tank
            mid_tank = Tank(d_i, d_o, V_tank, LF)
            
            # Set cryogen
            mid_tank.cryogen = Cryogen(name = Compound)
            
            mid_tank.cryogen.set_coolprops(P)
            
            q_b = mult*U_L*(T_air-mid_tank.cryogen.T_sat) # Heat flux proportional to the T diff of the cryogen.
            
            Q_b=q_b*mid_tank.A_T
            
            mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w)
            
            mid_tank.U_roof = U_V
            
            # Data table
            
            table = [[Compound, mid_tank.V, mid_tank.LF, f"{evap_time/3600} hrs", f"{res.fun*100}", f"{res.x[0]}",
                        P, T_air, mid_tank.d_i, mid_tank.d_o, mid_tank.l, U_L, U_V, eta_w, mid_tank.Q_b, q_b]]
            
            # structure the data table
            
            data_table = pd.DataFrame(table, columns = ["Cryogen", "Volume", "Liquid Filling","Simulation","BOR","Aspect Ratio"
                                                    , "Pressure (P)", "T_air (K)", "d_i", "d_o", "Height", "U_L", "U_V", "Eta_w","Q_b","q_b"])
            #print(Data_table)
            
            if sv:
                # Send Data table to excel
                # Try to load the existing Excel file
                try:
                    data_excel = pd.read_excel('Final.xlsx')
                    
                    # Append the row
                    data_excel = pd.concat([data_excel, data_table], ignore_index = True)
                    
                    # Save the updated DataFrame back to the Excel file
                    data_excel.to_excel('Final.xlsx', index = False)
                except:
                    #In case the file doesnt exist it is created
                    data_table.to_excel('Final.xlsx', index = False)
                print("\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\nData was saved.\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
            else:
                print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\nData was not saved.\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
                # Print a summary of the optimum if it is found
            if res.success:
                print(f"\nOptimum found for Tank Volume: {V_tank}, Liquid Filling: {LF*100}%, Compound: {Compound}" + 
                      f"\nAspect Ratio = {res.x[0]:.3f}" + f"\nBOR = {res.fun*100:.3f} %")


Optimum search for Tank Volume: 40000, Liquid Filling: 30.0%, Compound: hydrogen, time: 720.0 hrs


/tmp/ipykernel_192330/731150762.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("a = %.3f, BOR=%.3e" % (a, BOR))


a = 1.000, BOR=3.896e-02
a = 1.000, BOR=3.896e-02
a = 1.063, BOR=3.926e-02
a = 1.063, BOR=3.926e-02
a = 0.200, BOR=4.186e-02
a = 0.200, BOR=4.186e-02
a = 0.200, BOR=4.186e-02
a = 0.200, BOR=4.186e-02
a = 1.063, BOR=3.926e-02
a = 1.063, BOR=3.926e-02
a = 1.063, BOR=3.926e-02
a = 1.063, BOR=3.926e-02
a = 1.063, BOR=3.926e-02
a = 1.063, BOR=3.926e-02
a = 0.105, BOR=5.122e-02
a = 0.105, BOR=5.122e-02
a = 1.041, BOR=3.916e-02
a = 1.041, BOR=3.916e-02
a = 1.021, BOR=3.906e-02
a = 1.021, BOR=3.906e-02
a = 0.217, BOR=4.113e-02
a = 0.217, BOR=4.113e-02
a = 0.952, BOR=3.873e-02
a = 0.952, BOR=3.873e-02
a = 0.892, BOR=3.846e-02
a = 0.892, BOR=3.846e-02
a = 0.447, BOR=3.751e-02
a = 0.447, BOR=3.751e-02
a = 0.671, BOR=3.761e-02
a = 0.671, BOR=3.761e-02
a = 0.588, BOR=3.743e-02
a = 0.588, BOR=3.743e-02
a = 0.528, BOR=3.739e-02
a = 0.528, BOR=3.739e-02
a = 0.533, BOR=3.739e-02
a = 0.533, BOR=3.739e-02
a = 0.531, BOR=3.739e-02
a = 0.531, BOR=3.739e-02
a = 0.530, BOR=3.739e-02
a = 0.530, BOR=3.739e-02


/tmp/ipykernel_192330/731150762.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("a = %.3f, BOR=%.3e" % (a, BOR))


a = 1.000, BOR=2.601e-03
a = 1.000, BOR=2.601e-03
a = 1.066, BOR=2.623e-03
a = 1.066, BOR=2.623e-03
a = 0.105, BOR=3.413e-03
a = 0.105, BOR=3.413e-03
a = 1.044, BOR=2.616e-03
a = 1.044, BOR=2.616e-03
a = 1.022, BOR=2.608e-03
a = 1.022, BOR=2.608e-03
a = 0.116, BOR=3.289e-03
a = 0.116, BOR=3.289e-03
a = 0.116, BOR=3.289e-03
a = 0.116, BOR=3.289e-03
a = 0.996, BOR=2.600e-03
a = 0.996, BOR=2.600e-03
a = 0.972, BOR=2.592e-03
a = 0.972, BOR=2.592e-03
a = 0.218, BOR=2.737e-03
a = 0.218, BOR=2.737e-03
a = 0.903, BOR=2.571e-03
a = 0.903, BOR=2.571e-03
a = 0.844, BOR=2.553e-03
a = 0.844, BOR=2.553e-03
a = 0.398, BOR=2.518e-03
a = 0.398, BOR=2.518e-03
a = 0.664, BOR=2.509e-03
a = 0.664, BOR=2.509e-03
a = 0.610, BOR=2.501e-03
a = 0.610, BOR=2.501e-03
a = 0.516, BOR=2.495e-03
a = 0.516, BOR=2.495e-03
a = 0.539, BOR=2.495e-03
a = 0.539, BOR=2.495e-03
a = 0.530, BOR=2.495e-03
a = 0.530, BOR=2.495e-03
a = 0.528, BOR=2.495e-03
a = 0.528, BOR=2.495e-03
a = 0.528, BOR=2.495e-03
a = 0.528, BOR=2.495e-03


...

a = 1.000, BOR=3.686e-03
a = 1.000, BOR=3.686e-03
a = 1.067, BOR=3.662e-03
a = 1.067, BOR=3.662e-03
a = 1.380, BOR=3.605e-03
a = 1.380, BOR=3.605e-03
a = 1.505, BOR=3.598e-03
a = 1.505, BOR=3.598e-03
a = 1.572, BOR=3.597e-03
a = 1.572, BOR=3.597e-03
a = 1.583, BOR=3.597e-03
a = 1.583, BOR=3.597e-03
a = 1.582, BOR=3.597e-03
a = 1.582, BOR=3.597e-03
`gtol` termination condition is satisfied.
Number of iterations: 16, function evaluations: 14, CG iterations: 6, optimality: 4.71e-10, constraint violation: 0.00e+00, execution time:  3.7 s.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Data was saved.
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Optimum found for Tank Volume: 40000, Liquid Filling: 30.0%, Compound: hydrogen
Aspect Ratio = 1.582
BOR = 0.360 %

Optimum search for Tank Volume: 40000, Liquid Filling: 95.0%, Compound: hydrogen, time: 720.0 hrs


/tmp/ipykernel_192330/731150762.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("a = %.3f, BOR=%.3e" % (a, BOR))


a = 1.000, BOR=1.814e-02
a = 1.000, BOR=1.814e-02
a = 1.068, BOR=1.804e-02
a = 1.068, BOR=1.804e-02
a = 1.358, BOR=1.783e-02
a = 1.358, BOR=1.783e-02
a = 1.453, BOR=1.782e-02
a = 1.453, BOR=1.782e-02
a = 1.483, BOR=1.782e-02
a = 1.483, BOR=1.782e-02
a = 1.480, BOR=1.782e-02
a = 1.480, BOR=1.782e-02
a = 1.480, BOR=1.782e-02
a = 1.480, BOR=1.782e-02
a = 1.480, BOR=1.782e-02
a = 1.480, BOR=1.782e-02
a = 1.480, BOR=1.782e-02
a = 1.480, BOR=1.782e-02
`gtol` termination condition is satisfied.
Number of iterations: 18, function evaluations: 18, CG iterations: 8, optimality: 1.51e-09, constraint violation: 0.00e+00, execution time: 1.4e+02 s.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Data was saved.
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Optimum found for Tank Volume: 40000, Liquid Filling: 95.0%, Compound: hydrogen
Aspect Ratio = 1.480
BOR = 1.782 %

Optimum search for Tank Volume: 40000, Liquid Filling: 95.0%, Compound: hydrogen, time: 48.0 hrs


/tmp/ipykernel_192330/731150762.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("a = %.3f, BOR=%.3e" % (a, BOR))


a = 1.000, BOR=1.210e-03
a = 1.000, BOR=1.210e-03
a = 1.067, BOR=1.203e-03
a = 1.067, BOR=1.203e-03
a = 1.335, BOR=1.190e-03
a = 1.335, BOR=1.190e-03
a = 1.432, BOR=1.189e-03
a = 1.432, BOR=1.189e-03
a = 1.475, BOR=1.189e-03
a = 1.475, BOR=1.189e-03
a = 1.479, BOR=1.189e-03
a = 1.479, BOR=1.189e-03
a = 1.479, BOR=1.189e-03
a = 1.479, BOR=1.189e-03
a = 1.478, BOR=1.189e-03
a = 1.478, BOR=1.189e-03
a = 1.479, BOR=1.189e-03
a = 1.479, BOR=1.189e-03
`gtol` termination condition is satisfied.
Number of iterations: 19, function evaluations: 18, CG iterations: 8, optimality: 2.81e-09, constraint violation: 0.00e+00, execution time: 1.7e+01 s.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Data was saved.
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Optimum found for Tank Volume: 40000, Liquid Filling: 95.0%, Compound: hydrogen
Aspect Ratio = 1.479
BOR = 0.119 %
